# Get data (use AOI of water_body app)
This notebook uses the **PySTAC Client** (```pystac-client```, https://pystac-client.readthedocs.io/en/latest/) Python package to find suitable Sentinel-2 MSIL2A data by using a query with a number of defined paramteres. 

The STAC API for AWS is available on the endpoint (https://earth-search.aws.element84.com/v1/collections/).

# Set-up

In [1]:
import pystac
from pystac_client import Client

from shapely.geometry import box, mapping, Polygon
import shapely.wkt

import os
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
# Access to Catalog 
URL = "https://earth-search.aws.element84.com/v1/" # target STAC API for AWS

headers = []

cat = Client.open(URL, headers=headers, ignore_conformance=True)
cat

id: earth-search-aws
title: Earth Search by Element 84
description: A STAC API of public datasets on AWS
type: Catalog
"conformsTo: ['https://api.stacspec.org/v1.0.0/core', 'https://api.stacspec.org/v1.0.0/collections', 'https://api.stacspec.org/v1.0.0/ogcapi-features', 'https://api.stacspec.org/v1.0.0/item-search', 'https://api.stacspec.org/v1.0.0-rc.3/ogcapi-features#fields', 'https://api.stacspec.org/v1.0.0-rc.2/ogcapi-features#sort', 'https://api.stacspec.org/v1.0.0-rc.2/ogcapi-features#query', 'https://api.stacspec.org/v1.0.0-rc.3/item-search#fields', 'https://api.stacspec.org/v1.0.0-rc.2/item-search#sort', 'https://api.stacspec.org/v1.0.0-rc.2/item-search#query', 'https://api.stacspec.org/v0.3.0/aggregation', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson']"
id: cop-dem-glo-30
title: Copernicus DEM GLO-30
"description: The Copernicus DEM is a Digital Surface Model (DSM) which represents the surface of the Earth including buildings, infrastructure and vegetation. GLO-30 Public provides limited worldwide coverage at 30 meters because a small subset of tiles covering specific countries are not yet released to the public by the Copernicus Programme."
"providers: European Space Agency (licensor) Sinergise (producer, processor) OpenTopography (host) AWS (host) Element 84 (processor)"
type: Collection
"item_assets: {'data': {'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Data', 'raster:bands': [{'sampling': 'point', 'data_type': 'float32', 'spatial_resolution': 30, 'unit': 'meter'}]}}"


## Define params
A couple of S2 images (taken from the Water Bodies app) are:
- "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20210713_0_L2A
- "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20220524_0_L2A

In [3]:
# Collection 
collections=["sentinel-2-l2a"], 

# Start and End dates
start_date_str = '2021-07-12'; start_date = datetime.fromisoformat(start_date_str)
stop_date_str = '2021-07-14'; stop_date = datetime.fromisoformat(stop_date_str)

# BBOX or AOI
# bbox='-121.399,39.834,-120.74,40.472' #aoi = WKT format --> bbox=shapely.wkt.loads(aoi).bounds 
aoi='POLYGON((-121.73 40.52,-120.71 40.52,-120.71 39.79,-121.73 39.79,-121.73 40.52))'; bbox=shapely.wkt.loads(aoi).bounds 

# Other metadata
product_id = 'S2B_MSIL2A_20210713T184919_N0301_R113_T10TFK_20210713T213143'
cloud_cover = 30

# Define EPSG code
epsg = 'EPSG:4326'

## Query the Catalog

In [4]:
aoi2 = 'POLYGON((21.067 38.438,21.124 38.438,21.124 38.387,21.067 38.387,21.067 38.438))'
bbox=shapely.wkt.loads(aoi2).bounds 
bbox

(21.067, 38.387, 21.124, 38.438)

In [10]:
# Query by Product ID
query = cat.search(
    collections=collections, 
    # sortby="properties.datetime",
    ids = ['S2A_10TFK_20220524_0_L2A','S2B_10TFK_20210713_0_L2A']
)

In [8]:
# # Query by AOI, start and end date and other params 
# query = cat.search(
#     collections=collections, 
#     datetime = (start_date, stop_date),
#     # sortby="properties.datetime",
#     bbox=bbox, 
#     # query = {
#     #     'eo:cloud_cover': {'lt': 15}, # cloud_cover less than 15 percent by default
#     #     # 'sentinel:product_id': {'eq': product_id}
#     # }
# )

## Create multi-dimensional array with ```stackstac```

In [11]:
# items_stack = query.get_all_items()
items = query.get_all_items()

print(f'There are {len(items)} items in the collection')
display(items)

There are 2 items in the collection


id: S2A_10TFK_20220524_0_L2A
"bbox: [-121.8343226741975, 39.635880717283825, -120.51956282559038, 40.64479052153661]"
created: 2022-11-06T13:59:01.745Z
platform: sentinel-2a
constellation: sentinel-2
instruments: ['msi']
eo:cloud_cover: 2.249588
proj:epsg: 32610
mgrs:utm_zone: 10
mgrs:latitude_band: T
mgrs:grid_square: FK


In [12]:
# Display properties of one item 
items[0].properties

{'created': '2022-11-06T13:59:01.745Z',
 'platform': 'sentinel-2a',
 'constellation': 'sentinel-2',
 'instruments': ['msi'],
 'eo:cloud_cover': 2.249588,
 'proj:epsg': 32610,
 'mgrs:utm_zone': 10,
 'mgrs:latitude_band': 'T',
 'mgrs:grid_square': 'FK',
 'grid:code': 'MGRS-10TFK',
 'view:sun_azimuth': 142.980591686894,
 'view:sun_elevation': 67.0914247762071,
 's2:degraded_msi_data_percentage': 0,
 's2:nodata_pixel_percentage': 2e-05,
 's2:saturated_defective_pixel_percentage': 0,
 's2:dark_features_percentage': 0.443997,
 's2:cloud_shadow_percentage': 0,
 's2:vegetation_percentage': 49.610496,
 's2:not_vegetated_percentage': 43.638527,
 's2:water_percentage': 1.733807,
 's2:unclassified_percentage': 0.308692,
 's2:medium_proba_clouds_percentage': 0.186794,
 's2:high_proba_clouds_percentage': 0.00151,
 's2:thin_cirrus_percentage': 2.061284,
 's2:snow_ice_percentage': 2.014894,
 's2:product_type': 'S2MSI2A',
 's2:processing_baseline': '04.00',
 's2:product_uri': 'S2A_MSIL2A_20220524T18492

In [13]:
# print id of the first item
index = 0
item_id = items[index].id 
print(f'id: {item_id}')
date = items[index].datetime
print(f'date: {date}')

# print available bands
print(f'bands: {list(items[index].assets.keys())}')

id: S2A_10TFK_20220524_0_L2A
date: 2022-05-24 19:03:29.762000+00:00
bands: ['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2']
